In [11]:
def cu_resistivity_temp(t): 
    rho_r = 1.68e-8 *(1 + .00386 *(t - 293))
    return rho_r

def R_cu_clamps(t): 
    #R = cu_resistivity_temp(t) * (.03757/2)/( (.05064* .03000 - (3.14159* (.022225/2)**2)/2) ) 
    
    R = cu_resistivity_temp(t) * (.03)/    (3.14159* .022225/10)
    
    print(R)
    return R

I =  30/R_cu_clamps(473)
print(I)

1.22336968995062e-07
245224319.73290852


In [8]:
# Constants
g = 9.81 # m^2/s

# Cable geometry
D_can = .06351 
D_lid = .110

L_can = .167
L_lid = .017*2

#Stainless steel properties 
rho_m_ss = 7750 #kg/m^3 depending on alloy of stainless steel 
#rho_r_ss = 6.9e-7 #ohm-m
C_ss = 470 # J/kg-K
k_ss = 15 #W/m-K

# Air properties
rho_air = 1.204 # kg/m^3
T_air = 20.0 + 273.15 # K, ~20C room temp
nu_air = 1.516e-5 # m^2/s, kinematic visc
k_air = 2.514e-2 # W/m-K, thermal cond.
beta_air = 3.69e-3 # K^-1, thermal exp. coeff., compare to 1/T ~3.4e-3
Pr_air = 0.7 # [], ranges 0.69-0.74. Not very sensitive


T_s = 200.0 + 273.15 # K, surface temp

#calculation of convection losses to air at 200 C per second
Gr_473 = Grashof_cylinder(D, g, beta_air, T_s - T_air, nu_air)
Ra_473 = Gr_473 * Pr_air
h_473 = natural_convection_horiz_cyl(D_can, k_air, Pr_air, Ra_473)
A_surface_473 = 2*(np.pi * (D_lid)/2**2) + 2* (np.pi * D_lid * L_lid) + (np.pi* D_can *L_can)
P_conv_473 = h_473 * A_surface_473 *(T_s - T_air) #previously np.pi*D*L for the area. heat transfered per unit time


def P_conv_per_degree(t0, t, l, cool_heat):
    if cool_heat == "c": 
        T_s = np.arange(t0, t,  -1) #array so that we can calculate the power losses at each temperature
        #T_air = np.arange(t0/2 +10, t/2 +10, - ( (t0/2+10) - (t/2+10) )/  (t0-t) ) #creates some arbitrary cooling rate of air around it
        T_air = 293 * np.ones(abs(t0-t)) #assuming air around stays at room temp the whole time. worst case scenario
        
    if cool_heat == "h": 
        T_s = np.arange(t0, t, 1) #array so that we can calculate the power losses at each temperature
        
        #T_air = np.arange(t0, t/3, (t/3-t0)/(t-t0)) #some arbitrary heating rate of air around it 
        T_air = 293 * np.ones(abs(t0-t)) #assuming air around stays at room temp the whole time. worst case scenario. 
    
    Gr_temp = np.zeros(abs(t0-t))
 
    for i in range(len(T_s)): 
        Gr_temp[i] = Grashof_cylinder(D, g, beta_air, T_s[i] - T_air[i], nu_air)

    Ra = Gr_temp * Pr_air
    h = natural_convection_horiz_cyl(D, k_air, Pr_air, Ra)
    A_surface = (np.pi* D**4/4 *2) + (2* np.pi* D *l)
    
    P_conv = h * A_surface *(T_s - T_air) #previously np.pi*D*L for the area. heat transfered per unit time
    #print("P_conv per degree", P_conv)
    return P_conv

def iron_heat_capacity_temp(t0, t, cool_heat): 
    #the if statement is done because order matters in the array 
    if cool_heat == "c": 
        T = np.arange(t0, t, -1)
    elif cool_heat == "h": 
        T = np.arange(t0, t, 1)
        
    heat_capacity = np.zeros(abs(t0-t))
    
    for i in range(len(T)):
        heat_capacity[i] = 18.42868 + 24.64301*(T[i]/1000) - 8.913720*(T[i]/1000)**2 + 9.664706*(T[i]/1000)**3 -.012643/(T[i]/1000)**2
    
    heat_capacity = heat_capacity/ .055845 #from mol to kg
    return heat_capacity 

def chromium_heat_capacity_temp(t0, t, cool_heat): 
    if cool_heat == "c": 
        T = np.arange(t0, t, -1)
    elif cool_heat == "h": 
        T = np.arange(t0, t, 1)
        
    heat_capacity = np.zeros(abs(t0-t))
    
    for i in range(len(T)):
        heat_capacity[i] = 7.489737 + 71.50498*(T[i]/1000) - 91.67562*(T[i]/1000)**2 + 46.04450*(T[i]/1000)**3 + .138157/(T[i]/1000)**2
    
    heat_capacity = heat_capacity/.05194 #from mol to kg 
    return heat_capacity
    
def stainless_steel_heat_capacity_temp(t0, t, cool_heat): 
    heat_capacity = .1*chromium_heat_capacity_temp(t0, t, cool_heat) + .9*iron_heat_capacity_temp(t0, t, cool_heat)
    
    return heat_capacity


def Q_per_degree_for_totalT(t0, t, l, cool_heat): 
    """
    returns array of Q values. Each Q is the Q necessary to change material one degree (heat capacity). The array 
    has values for Q for the entire range t0 to t
    """
    
    C_stainless_steel_per_T = stainless_steel_heat_capacity_temp(t0, t, cool_heat)
    m_stainless_steel = rho_m_ss * ( 2*(np.pi * (D_lid)/2**2 * np.pi * D_lid * L_lid) + (np.pi* D_can *L_can * .020)
    Q =  m_stainless_steel * stainless_steel_heat_capacity_temp(t0, t, cool_heat))* 1
    return Q


def time_to_temp(t0, t, l, cool_heat, insulated=None):
    """
    heat_loss_rate should be ann array with entries with units J/s
    Assumes worst case scenario with heat capacity staying constant at the highest level it will be when it is at t
    
    """
    
    if cool_heat == "c":
        time_per_degree = np.divide(Q_per_degree_for_totalT(t0, t, l, cool_heat), P_conv_per_degree(t0, t, l, cool_heat))
        #print("time per degree", time_per_degree)
        total_time = sum(time_per_degree)
        return total_time

    elif cool_heat == "h": 
    
        I_at_each_temp_list = []
        Vs_0 = Vs_max_0 * np.ones(abs(t0-t))
              
        for temp in range(t0, t): 
            I_at_each_temp_list.append(Vs_max_0 / R_total(temp, l))
        I_at_each_temp = np.asarray(I_at_each_temp_list) 

        
        I_lim_ind_0 = np.where(I_at_each_temp > I_max)
  
      
        I_at_each_temp[I_lim_ind_0] = I_max

        Vs_0[I_lim_ind_0] = I_at_each_temp[I_lim_ind_0] * R_total(temp, l)
              
        P_electrical_at_each_temp = I_at_each_temp * Vs_0
        #print("electrical", P_electrical_at_each_temp)
        #print("P conv", P_conv_per_degree(t0, t, l, cool_heat))
        if insulated == "n": 
            P_net = P_electrical_at_each_temp - P_conv_per_degree(t0, t, l, cool_heat)
        elif insulated == "y": 
            #print("print this one")
            P_net = P_electrical_at_each_temp
        #print("Net", P_net)
        #print("net power need", P_net)
        #plt.plot(L, q)
        #print("electrical",P_electrical_at_each_temp)
        #print("conv", P_conv_per_degree(t0, t, l, cool_heat))
        #print("Q", Q_per_degree_for_totalT(t0,t, l, cool_heat))
        time_per_degree = np.divide(Q_per_degree_for_totalT(t0, t, l, cool_heat), P_net)
        total_time = sum(time_per_degree)
        #print("total time", total_time)
        return total_time
        
    else:
        return TypeError 

NameError: name 'Grashof_cylinder' is not defined